In [1]:
import pandas as pd
import warnings
from tabulate import tabulate
warnings.filterwarnings('ignore')
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [2]:
path = "C:/Users/gunheehan/Desktop/생산계획"

# Batch MPS 함수

In [3]:
def MPS(data, safety_stock, lot_size=0):
    data = data.fillna(0)
    max_lot = max (max (data["Forecast"]), max(data["Orders"]) )
    if lot_size == 0 :
        lot_size = max_lot
    for i in range(0,len(data)-1):
        data["Available(ending)"][i+1]=data["Available(ending)"][i]-max(data["Orders"][i+1],data["Forecast"][i+1])
        if data["Available(ending)"][i+1] <  safety_stock:
            data["MPS"][i+1]= lot_size
        else:
            data["MPS"][i+1]=0
        data["Available(ending)"][i+1]=data["Available(ending)"][i]-max(data["Orders"][i+1],data["Forecast"][i+1])+data["MPS"][i+1]
        
        
    for i in range(1,len(data)):
        if i==1:
            if data["Orders"][i]==0:
                data["ATP"][i]=data["Available(ending)"][0] 
            else:
                a=data["Orders"][i]
                for j in range(i+1,len(data)-1):
                    if data["MPS"][j] ==0:
                        a += data["Orders"][j]
                    else:
                        break
                
                data["ATP"][i]=data["Available(ending)"][0] + data["MPS"][i] -a
        else:
            if data["MPS"][i]==0:
                data["ATP"][i]=0
            else:
                a=data["Orders"][i]
                for j in range(i+1,len(data)):
                    if data["MPS"][j] ==0:
                        a += data["Orders"][j]
                    else:
                        break
                
                data["ATP"][i]= data["MPS"][i] -a
                
    return data

# chasing MPS 함수

In [4]:
def chasing_MPS(data):
    
    mps_df = pd.DataFrame(columns = ['week','Forecast','Order','Available(ending)','ATP','MPS'])
    mps_df['week'] = data['week']
    mps_df['Forecast']=data['Forecast']
    mps_df['Order'] = data['Orders']
    mps_df['Available(ending)'] = data['Available(ending)']
    
    for i in range(1,13):
        
        mps_df['Available(ending)'][i] = mps_df['Available(ending)'][i-1] - max(mps_df['Forecast'][i], mps_df['Order'][i])
        
        if mps_df['Available(ending)'][i]<0:
            mps_df['Available(ending)'][i] = 0 
        else:
            pass

        mps_df['MPS'][i] = mps_df['Forecast'][i] - mps_df['Available(ending)'][i-1]
        
        if mps_df['MPS'][i] < 0:
            mps_df['MPS'][i] = 0
        else:
            pass

        mps_df['ATP'][i] = mps_df['MPS'][i] - mps_df['Order'][i]
        
        if mps_df['ATP'][i] < 0:
            mps_df['ATP'][i] = 0
        else:
            pass
        
    return mps_df

In [5]:
df_a= pd.read_excel(path + '/a_input.xlsx')
df_b= pd.read_excel(path + '/b_input.xlsx')
df_c= pd.read_excel(path + '/c_input.xlsx')
MPS_a, MPS_b, MPS_c = MPS(df_a, 10, 120), MPS(df_b, 10, 120), MPS(df_c, 10, 120) # 수정 가능

# input 지정

In [6]:
input_file = "C:/Users/gunheehan/Desktop/생산계획/생계_input/d_input.xlsx"

# MRP of A, B, C

In [7]:
def mrp_abc(MPS, lead_time) :
    mrp = pd.read_excel(input_file)
    mrp = mrp.fillna(0)
    #lead time 설정
    mrp["Gross Requirement"] = MPS["MPS"]
    for idx, i in enumerate(mrp["Gross Requirement"]) : 
        if i != 0 :
            if idx - lead_time <= 0 :
                mrp["Scheduled Receipts"][idx] = i
            else :
                mrp["Planned Receipts"][idx] = i
                mrp["Planned Order Release"][idx - lead_time] = i
    return mrp

# MRP of D, J

In [8]:
def mrp_dj(MRP_a, inventory,  safety_stock = 0, batch = 0) :
    mrp = pd.read_excel(input_file)
    mrp = mrp.fillna(0)
    mrp["Gross Requirement"] = MRP_a["Planned Order Release"]
    
    mrp["PIB"][0] = inventory
    
    lead_time = 1
    for idx, i in enumerate(mrp["Gross Requirement"]) : 
        if idx == 0 :
            continue
        if i == 0 :
            mrp["PIB"][idx] = mrp["PIB"][idx-1]
        else :
            if mrp["PIB"][idx-1] - i >= safety_stock :
                mrp["PIB"][idx] = mrp["PIB"][idx-1] - i
            else :
                if idx - lead_time <= 0 :
                    mrp["Scheduled Receipts"][idx] = i - mrp["PIB"][idx]
                if batch != 0 :
                    mrp["PIB"][idx] = mrp["PIB"][idx-1] + batch - i
                    mrp["Planned Receipts"][idx] = batch
                else :  
                    mrp["PIB"][idx] = 0
                mrp["Net Requirements"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Receipts"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Order Release"][idx - lead_time] = mrp["Planned Receipts"][idx]
    return mrp

# MRP of E

In [9]:
def mrp_e(MRP_a, MRP_b, inventory,  safety_stock = 0, batch = 0) :
    mrp = pd.read_excel(input_file)
    mrp = mrp.fillna(0)
    mrp["Gross Requirement"] = MRP_a["Planned Order Release"] * 2 + MRP_b["Planned Order Release"] * 2
    mrp["PIB"][0] = inventory
    lead_time = 1
    for idx, i in enumerate(mrp["Gross Requirement"]) : 
        if idx == 0 :
            continue
        if i == 0 :
            mrp["PIB"][idx] = mrp["PIB"][idx-1]
        else :
            if mrp["PIB"][idx-1] - i >= safety_stock :
                mrp["PIB"][idx] = mrp["PIB"][idx-1] - i
            else :
                if idx - lead_time <= 0 :
                    mrp["Scheduled Receipts"][idx] = i - mrp["PIB"][idx]
                if batch != 0 :
                    mrp["PIB"][idx] = mrp["PIB"][idx-1] + batch - i
                    mrp["Planned Receipts"][idx] = batch
                else :  
                    mrp["PIB"][idx] = 0
                mrp["Net Requirements"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Receipts"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Order Release"][idx - lead_time] = mrp["Planned Receipts"][idx]
    return mrp

# MRP of F

In [10]:
def mrp_f(MRP_d, MRP_h, inventory,  safety_stock = 0, batch = 0) :
    mrp = pd.read_excel(input_file)
    mrp = mrp.fillna(0)
    mrp["Gross Requirement"] = MRP_d["Planned Order Release"] * 3 + MRP_h["Planned Order Release"] * 2
    mrp["PIB"][0] = inventory
    lead_time = 2
    for idx, i in enumerate(mrp["Gross Requirement"]) : 
        if idx == 0 :
            continue
        if i == 0 :
            mrp["PIB"][idx] = mrp["PIB"][idx-1]
        else :
            if mrp["PIB"][idx-1] - i >= safety_stock :
                mrp["PIB"][idx] = mrp["PIB"][idx-1] - i
            else :
                if idx - lead_time <= 0 :
                    mrp["Scheduled Receipts"][idx] = i - mrp["PIB"][idx]
                if batch != 0 :
                    mrp["PIB"][idx] = mrp["PIB"][idx-1] + batch - i
                    mrp["Planned Receipts"][idx] = batch
                else :  
                    mrp["PIB"][idx] = 0
                mrp["Net Requirements"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Receipts"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Order Release"][idx - lead_time] = mrp["Planned Receipts"][idx]
    return mrp

# MRP of G

In [11]:
def mrp_g(MRP_d, MRP_c, inventory,  safety_stock = 0, batch = 0) :
    mrp = pd.read_excel(input_file)
    mrp = mrp.fillna(0)
    mrp["Gross Requirement"] = MRP_d["Planned Order Release"] + MRP_c["Planned Order Release"] * 2
    mrp["PIB"][0] = inventory
    lead_time = 1
    for idx, i in enumerate(mrp["Gross Requirement"]) : 
        if idx == 0 :
            continue
        if i == 0 :
            mrp["PIB"][idx] = mrp["PIB"][idx-1]
        else :
            if mrp["PIB"][idx-1] - i >= safety_stock :
                mrp["PIB"][idx] = mrp["PIB"][idx-1] - i
            else :
                if idx - lead_time <= 0 :
                    mrp["Scheduled Receipts"][idx] = i - mrp["PIB"][idx]
                if batch != 0 :
                    mrp["PIB"][idx] = mrp["PIB"][idx-1] + batch - i
                    mrp["Planned Receipts"][idx] = batch
                else :  
                    mrp["PIB"][idx] = 0
                mrp["Net Requirements"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Receipts"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Order Release"][idx - lead_time] = mrp["Planned Receipts"][idx]
    return mrp

# MRP of H

In [12]:
def mrp_h(MRP_b, MRP_c, inventory,  safety_stock = 0, batch = 0) :
    mrp = pd.read_excel(input_file)
    mrp = mrp.fillna(0)
    mrp["Gross Requirement"] = MRP_b["Planned Order Release"] + MRP_c["Planned Order Release"] * 2
    mrp["PIB"][0] = inventory
    lead_time = 2
    for idx, i in enumerate(mrp["Gross Requirement"]) : 
        if idx == 0 :
            continue
        if i == 0 :
            mrp["PIB"][idx] = mrp["PIB"][idx-1]
        else :
            if mrp["PIB"][idx-1] - i >= safety_stock :
                mrp["PIB"][idx] = mrp["PIB"][idx-1] - i
            else :
                if idx - lead_time <= 0 :
                    mrp["Scheduled Receipts"][idx] = i - mrp["PIB"][idx]
                if batch != 0 :
                    mrp["PIB"][idx] = mrp["PIB"][idx-1] + batch - i
                    mrp["Planned Receipts"][idx] = batch
                else :  
                    mrp["PIB"][idx] = 0
                mrp["Net Requirements"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Receipts"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Order Release"][idx - lead_time] = mrp["Planned Receipts"][idx]
    return mrp

# MRP of I

In [13]:
def mrp_i(MRP_h, inventory,  safety_stock = 0, batch = 0) :
    mrp = pd.read_excel(input_file)
    mrp = mrp.fillna(0)
    mrp["Gross Requirement"] = MRP_h["Planned Order Release"] * 3
    mrp["PIB"][0] = inventory
    lead_time = 1
    for idx, i in enumerate(mrp["Gross Requirement"]) : 
        if idx == 0 :
            continue
        if i == 0 :
            mrp["PIB"][idx] = mrp["PIB"][idx-1]
        else :
            if mrp["PIB"][idx-1] - i >= safety_stock :
                mrp["PIB"][idx] = mrp["PIB"][idx-1] - i
            else :
                if idx - lead_time <= 0 :
                    mrp["Scheduled Receipts"][idx] = i - mrp["PIB"][idx]
                if batch != 0 :
                    mrp["PIB"][idx] = mrp["PIB"][idx-1] + batch - i
                    mrp["Planned Receipts"][idx] = batch
                else :  
                    mrp["PIB"][idx] = 0
                mrp["Net Requirements"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Receipts"][idx] = i - mrp["PIB"][idx-1]
                mrp["Planned Order Release"][idx - lead_time] = mrp["Planned Receipts"][idx]
    return mrp

In [20]:
def print_all(path) :
    info_data = pd.read_excel(path + '/data_input.xlsx')
    info = info_data.rename(index={0: 'A', 1: 'B', 2: 'C', 3 : 'D', 4 : 'E', 5: 'F', 6: 'G', 7: 'H', 8:'I' ,9:'J'} )
    
    df_a= pd.read_excel(path + '/a_input.xlsx')
    df_b= pd.read_excel(path + '/b_input.xlsx')
    df_c= pd.read_excel(path + '/c_input.xlsx')
    input_file = path + "/d_input.xlsx"
    if info["Chasing"]["A"] == "N" :  # Chasing 아닌 경우 = batch방식
        MPS_a = MPS(df_a, info["Safety Stock"]["A"], lot_size = info["Batch"]["A"])
        MPS_b = MPS(df_b, info["Safety Stock"]["B"], lot_size = info["Batch"]["B"])
        MPS_c = MPS(df_c, info["Safety Stock"]["C"], lot_size = info["Batch"]["C"])
    else :
        MPS_a = MPS(df_a)
        MPS_b = MPS(df_b)
        MPS_c = MPS(df_c)
    
    print(color.BOLD + "MPS of A" + color.END)
    print(tabulate(MPS_a, headers='keys', tablefmt='pretty', showindex=False))
    
    print(color.BOLD + "MPS of B" + color.END)
    print(tabulate(MPS_b, headers='keys', tablefmt='pretty', showindex=False))
    
    print(color.BOLD + "MPS of C" + color.END)
    print(tabulate(MPS_c, headers='keys', tablefmt='pretty', showindex=False))
    
    input_file = path + "/d_input.xlsx"
    
    #MRP of A,B,C
    MRP_a = mrp_abc(MPS_a, 2)
    MRP_b = mrp_abc(MPS_b, 1)
    MRP_c = mrp_abc(MPS_c, 1)
    
    #MRP of D, J
    MRP_d = mrp_dj(MRP_a, info["Initial Inventory"]["D"],  safety_stock = info["Safety Stock"]["D"], batch = info["Batch"]["D"])
    MRP_j = mrp_dj(MRP_c, info["Initial Inventory"]["J"],  safety_stock = info["Safety Stock"]["J"], batch = info["Batch"]["J"])
    
    #MRP of E
    MRP_e = mrp_e(MRP_a, MRP_b, info["Initial Inventory"]["E"],  safety_stock = info["Safety Stock"]["E"], batch = info["Batch"]["E"])
    
    #MRP of H
    MRP_h = mrp_h(MRP_b, MRP_c, info["Initial Inventory"]["H"],  safety_stock = info["Safety Stock"]["H"], batch = info["Batch"]["H"])
    
    #MRP of F
    MRP_f = mrp_f(MRP_d, MRP_h, info["Initial Inventory"]["F"],  safety_stock = info["Safety Stock"]["F"], batch = info["Batch"]["F"])
    
    #MRP of G
    MRP_g = mrp_g(MRP_d, MRP_c, info["Initial Inventory"]["G"],  safety_stock = info["Safety Stock"]["G"], batch = info["Batch"]["G"])
    
    #MRP of I
    MRP_i = mrp_i(MRP_h, info["Initial Inventory"]["I"],  safety_stock = info["Safety Stock"]["I"], batch = info["Batch"]["I"])
    
    print(color.BOLD + "MRP of A" + color.END)
    print(color.GREEN + tabulate(MRP_a, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of B" + color.END)
    print(color.GREEN + tabulate(MRP_b, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of C" + color.END)
    print(color.GREEN + tabulate(MRP_c, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of D" + color.END)
    print(color.GREEN + tabulate(MRP_d, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of E" + color.END)
    print(color.GREEN + tabulate(MRP_e, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of F" + color.END)
    print(color.GREEN + tabulate(MRP_f, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of G" + color.END)
    print(color.GREEN + tabulate(MRP_g, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of H" + color.END)
    print(color.GREEN + tabulate(MRP_h, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of I" + color.END)
    print(color.GREEN + tabulate(MRP_i, headers='keys', tablefmt='pretty', showindex=False) + color.END)
    print()
    
    print(color.BOLD + "MRP of J" + color.END)
    print(color.GREEN + tabulate(MRP_j, headers='keys', tablefmt='pretty', showindex=False) + color.END)

In [21]:
print_all("C:/Users/gunheehan/Desktop/생산계획/생계_input")

MPS of A
+------+----------+--------+-------------------+-------+-------+
| week | Forecast | Orders | Available(ending) |  ATP  |  MPS  |
+------+----------+--------+-------------------+-------+-------+
| 0.0  |   0.0    |  0.0   |       160.0       |  0.0  |  0.0  |
| 1.0  |  100.0   | 100.0  |       60.0        | 60.0  |  0.0  |
| 2.0  |   80.0   |  40.0  |       100.0       | 60.0  | 120.0 |
| 3.0  |   70.0   |  20.0  |       30.0        |  0.0  |  0.0  |
| 4.0  |   70.0   |  10.0  |       80.0        | 110.0 | 120.0 |
| 5.0  |  120.0   |  0.0   |       80.0        | 120.0 | 120.0 |
| 6.0  |  100.0   |  0.0   |       100.0       | 120.0 | 120.0 |
| 7.0  |  120.0   |  0.0   |       100.0       | 120.0 | 120.0 |
| 8.0  |   60.0   |  0.0   |       40.0        |  0.0  |  0.0  |
| 9.0  |   70.0   |  0.0   |       90.0        | 120.0 | 120.0 |
| 10.0 |   80.0   |  0.0   |       10.0        |  0.0  |  0.0  |
| 11.0 |  110.0   |  0.0   |       20.0        | 120.0 | 120.0 |
| 12.0 |  100.0 